# Dimensionality Reduction 1

Learning objectives
1. Apply PCA to different data sets and interpret the output
2. Learn how to visualise the model output
3. Interpret the results to learn about the data structure and potential outliers
4. Code your own function to perform scaling (centering and auto-scaling) using only the numpy.mean and numpy.std functions and two datasets as input (training, test)
5. Investigate the effect of scaling the data on the output

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler

Read in the omics datasets using the pandas [read_excel()](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) function. For this example we will be using some COVID19 proteomics data.

In [ ]:
covid_proteomics = pd.read_excel("../Data/COVID19_proteomics.xlsx")

In [ ]:
covid_proteomics.head(20)

We can see the data has three metadata colums: COVID19 (disease status), sample_time (when the blood draw was taken), and sample_id.

## Principal component analysis
Read more about PCA in the sklearn [documentation](https://scikit-learn.org/stable/modules/decomposition.html#pca)

Use the sklearn [PCA()](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html?highlight=pca#sklearn.decomposition.PCA) function to initialise a PCA object. What is the maximum number of components you can have for a PCA model?

In [ ]:
pca_res = PCA(n_components=4)
# run PCA with 4 components

Apply the `fit_transform()` function to the PCA object to perform PCA dimensionality reduction on the data and project the data to the latent space. Note: we must exclude any sample metadata columns beforehand. 

In [ ]:
# covid_proteomics.iloc[:, 3:] dataframe without the first 3 metadata columns
pca_covid = pca_res.fit_transform(covid_proteomics.iloc[:, 3:])

By appling `fit_transform` we project the proteomics data into the latent subspace captured by PCA. The results, also known as **PCA scores** are stored in the results of `fit_transform`, in our case the variable `pca_covid`. Let's look at the PCA results:

In [ ]:
pca_covid

In [ ]:
print(pca_covid.shape)
print(type(pca_covid))

The results are returned as a numpy array, in this case of size 382 rows (number of samples) by 4 columns (number of components we selected)

We can also obtain the PCA components, **also known as eigenvectors**, which represent the influence of each variable (in this case each protein) within each principal component. We do so using the `components_` attribute of the PCA results. There are 4 rows (components) and 450 columns (proteins).

In [ ]:
print(pca_res.components_)
print()
print(pca_res.components_.shape)

## PCA visualisation

Visutalise two sets of PCA scores against each other with a PCA biplot

In [ ]:
# set plotting parameters
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

# plot a scatterplot using seaborn
# the x axis will contain the first column of the pca scores x=pca_covid[:, 0]
p = sns.scatterplot(x=pca_covid[:, 0],
 y=pca_covid[:, 1])

p.set_xlabel("PC1")
p.set_ylabel("PC2")

plt.show()

Colour the scatterpoints on the biplot by some metadata - here we will use COVID19 status

In [ ]:
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=pca_covid[:, 0],
 y=pca_covid[:, 1], 
 hue=covid_proteomics["COVID19"])

p.set_xlabel("PC1")
p.set_ylabel("PC2")

plt.show()

Visualise multiple components against each other. Here we make use of the seaborn [pairplot()](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function to do so

In [ ]:
# make a dataframe to store the PCA scores for each component alongside the sample metadata
pca_df = pd.DataFrame(pca_covid, columns=["PC"+str(i) for i in range(1, pca_covid.shape[1]+1)])
pca_df["COVID_status"] = covid_proteomics["COVID19"]

pca_df

In [ ]:
sns.pairplot(data=pca_df,
 hue="COVID_status")

plt.show()

Scree plots show the percentage of the variance in the data explained by each principal component (eigenvalues)

In [ ]:
# Scree plot

# perform PCA with 20 components
pca_covid = PCA(n_components=20).fit(covid_proteomics.iloc[:, 3:])

# use the attribute .explained_variance_ratio_ to get the eigenvalues
variance_per_component = pca_covid.explained_variance_ratio_

# sum the eigenvalues to get the cumulative variance explained for each component
cumulative_variance = np.cumsum(variance_per_component)
components = list(range(1, 21))

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
sns.barplot(x=components, y=variance_per_component, palette="viridis", ax=ax)

# show the cumulative variance with a blue line
sns.pointplot(x=components, y=cumulative_variance, ax=ax, color="blue", label="Cumulative variance")

plt.xlabel("Components")
plt.ylabel("Eigenvalue (% variance explained)")
plt.show()

How many components would be required to explain 75% of the variance in the dataset?

### Interpretation of PCA results

Looking at the PCA bi-plot and the PCA pairplot, which PCA components show the most clear separation between COVID19 and non-COVID samples?

### Identifying outliers
Samples can be labelled in order to identify those that are outlying

In [ ]:
pca_covid = PCA(n_components=2).fit_transform(covid_proteomics.iloc[:, 3:])

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(x=pca_covid[:, 0], y=pca_covid[:, 1], hue=covid_proteomics["COVID19"])

# for loop to add labels to each x, y pair along with the corresponding sample ID
for i in range(pca_covid.shape[0]):
    plt.text(x=pca_covid[:, 0][i]+0.3, y=pca_covid[:, 1][i]+0.3, s=covid_proteomics["sample_id"][i], 
          fontdict=dict(color='black',size=8))

plt.tight_layout()
plt.show()

Are the any obvious outliers in this dataset as seen from this PCA biplot?

## Scaling


The standard score of a sample $x$ is defined as:


$$z = \frac{(x-\mu)}{\sigma}$$

Where:
- $\mu$ is the feature mean
- $\sigma$ is the feature standard deviation



Code a function to scale the data such that each feature has a:
- Mean of 0 
- Standard deviation of 1

Use only the numpy [mean()](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) and numpy [std()](https://numpy.org/doc/stable/reference/generated/numpy.std.html) functions

You can check your answer by comparing it to the result achieved using the sklearn [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#:~:text=Standardize%20features%20by%20removing%20the%20mean%20and%20scaling%20to%20unit%20variance.&text=where%20u%20is%20the%20mean,or%20one%20if%20with_std%3DFalse%20.) function

In [ ]:
def my_scaler(data):
    
    # get mean of each column

    # subtract the column mean from each value in the column

    # check the mean of each column is now 0

    # get the standard deviations of each column

    # divide the mean centered data by the standard deviation to scale by unit variance

    return scaled_data


In [ ]:
# apply the scaler function to the data
covid_proteomics_scaled = my_scaler(covid_proteomics.iloc[:, 3:])

In [ ]:
# perform PCA using the scaled data
pca_covid_scaled = PCA(n_components=2).fit_transform(covid_proteomics_scaled)

How does scaling affect the PCA? visualise the output

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

sns.scatterplot(x=pca_covid[:, 0], y=pca_covid[:, 1], hue=covid_proteomics["COVID19"], ax=ax1)
ax1.set_title("No scaling")

sns.scatterplot(x=pca_covid_scaled[:, 0], y=pca_covid_scaled[:, 1], hue=covid_proteomics["COVID19"], ax=ax2)
ax2.set_title("Standard scaling")

sklearn also has a number of inbuilt functions for scaling. Look into the following functions and apply them on the dataset:
- [sklearn.StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#:~:text=Standardize%20features%20by%20removing%20the%20mean%20and%20scaling%20to%20unit%20variance.&text=where%20u%20is%20the%20mean,or%20one%20if%20with_std%3DFalse%20.)
- [sklearn.RobustScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html?highlight=robust%20scaler#sklearn.preprocessing.RobustScaler)
- [sklearn.PowerTransformer()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html?highlight=power%20transformer#sklearn.preprocessing.PowerTransformer)

Let's begin with appling the PowerTransformer():

In [ ]:
# initialise PowerTransformer object
pt = PowerTransformer()

# Apply the power transformation to the data
covid_proteomics_power_transform = pt.fit_transform(covid_proteomics.iloc[:, 3:])

# Apply PCA to the power transformed data
pca_covid_power_transform = PCA(n_components=2).fit_transform(covid_proteomics_power_transform)

Complete the following code for the Standard Scaler and Robust Scaler:

In [ ]:
# Standard scaler
ss = StandardScaler()

covid_proteomics_standard_scaled = 

pca_covid_standard_scaled = 

# Robust scaler
rs = 

covid_proteomics_robust_scaled = 

pca_covid_robust_scaled = 

Visualise the results of the scaling using PCA biplots

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=pca_covid_power_transform[:, 0], y=pca_covid_power_transform[:, 1], hue=covid_proteomics["COVID19"], ax=ax1)
ax1.set_title("Power transformer")

sns.scatterplot(x=pca_covid_standard_scaled[:, 0], y=pca_covid_standard_scaled[:, 1], hue=covid_proteomics["COVID19"], ax=ax2)
ax2.set_title("Standard scaling")

sns.scatterplot(x=pca_covid_robust_scaled[:, 0], y=pca_covid_robust_scaled[:, 1], hue=covid_proteomics["COVID19"], ax=ax3)
ax3.set_title("Robust scaling")

plt.tight_layout()
plt.show()

## PCA extensions: Kernel PCA and sparse PCA

### Sparse PCA

Sparse PCA is performed using the [SparsePCA()](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html#sklearn.decomposition.SparsePCA) function. The level of sparsity (the proportion of input variables contributing to the principal components) is controllable by the coefficient of the L1 penalty, given by the parameter `alpha`.

In [ ]:
sparse_pca = SparsePCA(n_components=2, alpha=1)
sparse_pca_covid = sparse_pca.fit_transform(covid_proteomics.iloc[:, 3:])

Look at the mean number of 0 values across the components (sparsity level). How does this change when you change the alpha parameter? 

If `alpha` = 0 there is no sparsity constraint, and all input variables will contribute to the principal components. The higher alpha is, the less variables will contribute to the principal components. 

In [ ]:
np.mean(sparse_pca.components_ == 0)

In [ ]:
sparse_pca_1 = SparsePCA(n_components=2, alpha=1).fit_transform(covid_proteomics.iloc[:, 3:])
sparse_pca_10 = SparsePCA(n_components=2, alpha=10).fit_transform(covid_proteomics.iloc[:, 3:])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

sns.scatterplot(x=sparse_pca_1[:, 0], y=sparse_pca_1[:, 1], hue=covid_proteomics["COVID19"], ax=ax1)
ax1.set_title("alpha=1")

sns.scatterplot(x=sparse_pca_10[:, 0], y=sparse_pca_10[:, 1], hue=covid_proteomics["COVID19"], ax=ax2)
ax2.set_title("alpha=10")

plt.tight_layout()
plt.show()

### Kernel PCA

[Kernel PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html#sklearn.decomposition.KernelPCA) is a form of non-linear dimensionality reduction using kernels. There are several hyperparameters that can be tuned for this model, the main being the type of kernel used. The sklearn KernelPCA() function supports the following kernels: 'linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’, ‘precomputed’, with the default being ’linear’.

In [ ]:
# no kernel has been specified so it uses linear by default
kernel_pca = KernelPCA(n_components=2)
kernel_pca_covid = kernel_pca.fit_transform(covid_proteomics.iloc[:, 3:])

Now try to use another kernel type:

In [ ]:
kernel_pca_rbf = KernelPCA(n_components=2, kernel="rbf").fit_transform(covid_proteomics.iloc[:, 3:])

Experiment with the other parameters listed on the reference page, such as Gamma - the kernel bandwidth parameter.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

sns.scatterplot(x=kernel_pca_covid[:, 0], y=kernel_pca_covid[:, 1], hue=covid_proteomics["COVID19"], ax=ax1)
ax1.set_title("Linear kernel")

sns.scatterplot(x=kernel_pca_rbf[:, 0], y=kernel_pca_rbf[:, 1], hue=covid_proteomics["COVID19"], ax=ax2)
ax2.set_title("Radial basis function kernel")

plt.tight_layout()
plt.show()

Which kernel type do you think provides the best separation for this dataset?

Visualise the results of the COVID19 dataset using different PCA types:

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=pca_covid[:, 0], y=pca_covid[:, 1], hue=covid_proteomics["COVID19"], ax=ax1)
ax1.set_title("PCA")

sns.scatterplot(x=sparse_pca_covid[:, 0], y=sparse_pca_covid[:, 1], hue=covid_proteomics["COVID19"], ax=ax2)
ax2.set_title("Sparse PCA")

# if we use linear kernel PCA this will be the same result as standard PCA
sns.scatterplot(x=kernel_pca_covid[:, 0], y=kernel_pca_covid[:, 1], hue=covid_proteomics["COVID19"], ax=ax3)
ax3.set_title("Kernel PCA")

plt.tight_layout()
plt.show()

## Your turn
Select another dataset from the `Data` folder and import it using the pandas `read_excel()` function as above. Scale the data, and then apply PCA (standard, sparse, or kernel) and visualise the results.

What can you interpret from the PCA results? Are there any outliers?

In [ ]:
# Import datasets...

In [ ]:
# Perform scaling...

In [ ]:
# Perform PCA...

In [ ]:
# Visualise PCA results and detect outliers